In [1]:
import json
import torch
import matplotlib.pyplot as plt

In [1]:
import torch

data = torch.load("../checkpoints/last.ckpt")

In [6]:
data["optimizer_states"][0]["param_groups"][0]['lr'] = 0.00005

[{'lr': 0.0002,
  'betas': (0.9, 0.999),
  'eps': 1e-08,
  'weight_decay': 0.0001,
  'amsgrad': False,
  'foreach': None,
  'maximize': False,
  'capturable': False,
  'differentiable': False,
  'fused': None,
  'initial_lr': 0.0002,
  'params': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   101,
   102,
   103,
   104,
   105,
   106,
   107,
  

In [2]:
with open("../datasets/bdd100k/labels/bdd100k_labels_images_train.json") as f:
    data = json.load(f)

In [3]:
len(data)

69863

In [6]:
reduced_data = []

for image in data:
    reduced_data.append(
        {
            "name": image["name"],
            "attributes": {"timeofday": image["attributes"]["timeofday"]},
            "labels": [
                {"category": label["category"], "box2d": label["box2d"]}
                for label in image["labels"]
                if "box2d" in label and label["box2d"] is not None
            ],
        }
    )

with open(
    "../datasets/bdd100k/labels/bdd100k_labels_images_train_reduced.json", "w"
) as f:
    json.dump(reduced_data, f)

In [7]:
def display_image(image_name):
    plt.imshow(plt.imread("../dataset/bdd100k/images/100k/train/" + image_name))
    plt.show()

In [9]:
categorie_to_id = {
    "bike": 0,
    "bus": 1,
    "car": 2,
    "motor": 3,
    "person": 4,
    "rider": 5,
    "traffic light": 6,
    "traffic sign": 7,
    "train": 8,
    "truck": 9,
    "pedestrian": 10,
    "motorcycle": 11,
    "bicycle": 12,
}

In [10]:
day_images = set()
night_images = set()

for image in data:
    if image["attributes"]["timeofday"] == "daytime":
        day_images.add(image["name"])
    elif image["attributes"]["timeofday"] == "night":
        night_images.add(image["name"])

In [21]:
def to_tensor(images, score=False, weather=None, timeofday=None):
    sorted_images = sorted(images, key=lambda x: x["name"])

    results = []
    for img in sorted_images:
        if score:
            if timeofday == "daytime" and img["name"] not in day_images:
                continue
            if timeofday == "night" and img["name"] not in night_images:
                continue
        else:
            if weather is not None and img["attributes"]["weather"] != weather:
                continue
            if timeofday is not None and img["attributes"]["timeofday"] != timeofday:
                continue

        labels = img["labels"]
        labels = [l for l in labels if "box2d" in l and l["box2d"] is not None]
        boxes = [l["box2d"] for l in labels]
        boxes = [[b["x1"], b["y1"], b["x2"], b["y2"]] for b in boxes]
        boxes = torch.tensor(boxes)

        categories = [l["category"] for l in labels]
        categories = [categorie_to_id[c] for c in categories]
        categories = torch.tensor(categories)

        result = {"boxes": boxes, "labels": categories}

        if score:
            scores = [l["score"] for l in labels]
            scores = torch.tensor(scores)
            result["scores"] = scores

        results.append(result)

    return results

In [22]:
gt = to_tensor(data)
day = to_tensor(data, timeofday="daytime")
night = to_tensor(data, timeofday="night")
len(day), len(night)

(36728, 27971)

In [44]:
from torchmetrics.detection import MeanAveragePrecision

# randam dummy predictions
N = 10000
preds = []
for i in range(N):
    boxes = torch.randint(0, 100, (100, 4))
    labels = torch.randint(0, 10, (100,))
    scores = torch.rand(100)

    # to cuda
    boxes = boxes.cuda()
    labels = labels.cuda()
    scores = scores.cuda()

    preds.append({"boxes": boxes, "labels": labels, "scores": scores})

gt = []
for i in range(N):
    boxes = torch.randint(0, 100, (10, 4))
    labels = torch.randint(0, 10, (10,))

    # to cuda
    boxes = boxes.cuda()
    labels = labels.cuda()

    gt.append({"boxes": boxes, "labels": labels})

mAP = MeanAveragePrecision(box_format="cxcywh")
mAP.update(preds, gt)
mAP.cpu()
mAP.compute()

{'map': tensor(0.0002),
 'map_50': tensor(0.0010),
 'map_75': tensor(1.3722e-05),
 'map_small': tensor(1.9761e-06),
 'map_medium': tensor(0.0003),
 'map_large': tensor(0.0022),
 'mar_1': tensor(0.0021),
 'mar_10': tensor(0.0178),
 'mar_100': tensor(0.0200),
 'mar_small': tensor(0.0010),
 'mar_medium': tensor(0.0298),
 'mar_large': tensor(0.0587),
 'map_per_class': tensor(-1.),
 'mar_100_per_class': tensor(-1.),
 'classes': tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=torch.int32)}

In [24]:
def compute_map(baseline):
    with open(f"../baselines/{baseline}.json") as f:
        baseline = json.load(f)

    day_preds = to_tensor(baseline["frames"], score=True, timeofday="daytime")
    night_preds = to_tensor(baseline["frames"], score=True, timeofday="night")

    iou_thresholds = [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]
    day_map = MeanAveragePrecision("xyxy", "bbox", iou_thresholds=iou_thresholds)
    night_map = MeanAveragePrecision("xyxy", "bbox", iou_thresholds=iou_thresholds)

    print(len(day_preds), len(night_preds))

    day_map.update(day_preds, day)
    night_map.update(night_preds, night)

    return day_map.compute(), night_map.compute()

In [25]:
result = compute_map("cascade_rcnn_swin-t_fpn_3x_det_bdd100k")
print(result)

0 0


ValueError: Expected argument `preds` and `target` to have the same length, but got 0 and 36728

In [ ]:
print(
    "Daytime mAP: {} mAP_50: {}, mAP_75: {}".format(
        result[0]["map"], result[0]["map_50"], result[0]["map_75"]
    )
)
print(
    "Nighttime mAP: {} mAP_50: {}, mAP_75: {}".format(
        result[1]["map"], result[1]["map_50"], result[1]["map_75"]
    )
)

Daytime mAP: 0.2511175274848938 mAP_50: 0.41557979583740234, mAP_75: 0.25299593806266785
Nighttime mAP: 0.20862507820129395 mAP_50: 0.3685239851474762, mAP_75: 0.19639737904071808


In [ ]:
# ConvNeXt + R-CNN
# Daytime mAP: 0.26108455657958984 mAP_50: 0.42570289969444275, mAP_75: 0.26357346773147583
# Nighttime mAP: 0.20968283712863922 mAP_50: 0.36812451481819153, mAP_75: 0.19964323937892914

# RetinaNet
# Daytime mAP: 0.23603859543800354 mAP_50: 0.405560702085495, mAP_75: 0.23004977405071259
# Nighttime mAP: 0.19778987765312195 mAP_50: 0.3620649576187134, mAP_75: 0.1865830421447754